In [1]:
import pandas
import datetime
import numpy
import scipy.optimize
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
import bs4
import urllib
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
%matplotlib inline
from matplotlib import rcParams

In [2]:
# setup some cutoff values we'll use in the analysis
velCutoffUpper = 2000.
velCutoffLower = 0.
numPointsCutoffMLTMLAT = 250

In [5]:
velGmagDF = pandas.read_csv("../data/processed-vels-geomag.txt", sep=' ',\
                            parse_dates=["date", "dst_date"], infer_datetime_format=True)
velGmagDF = velGmagDF.drop('Unnamed: 0', axis=1)
# Filter out some values where number of datapoints are pretty low.
countDF = velGmagDF.groupby([ "normMLT", "MLAT" ]).size().reset_index()
countDF.columns = [ "normMLT", "MLAT", "count" ]
# Choose only columns which have atleast 100 points
countDF = countDF[ countDF["count"] >= numPointsCutoffMLTMLAT ].reset_index(drop=True)
# Merge with velsDataDF to filter out unwanted values
velGmagDF = pandas.merge( velGmagDF, countDF,\
                          on=["normMLT", "MLAT"], how='inner' )
velGmagDF.head()

,normMLT,MLAT,vSaps,azim,vMagnErr,azimErr,dtStr,tmStr,date,hour,dst_date,dst_index,dst_bin,AE,AL,AO,AU,minute,count
0,-4,60.5,634.32,-10.10,7.19,21.54,20130316,500,2013-03-16 05:00:00,5,2013-03-16 05:00:00,-19,"(-25, -10]",731,-575,-210,156,0,2677
1,-4,60.5,609.64,-10.92,3.58,9.73,20130316,502,2013-03-16 05:02:00,5,2013-03-16 05:00:00,-19,"(-25, -10]",651,-524,-199,127,2,2677
2,-4,60.5,613.71,-12.20,4.15,8.27,20130316,504,2013-03-16 05:04:00,5,2013-03-16 05:00:00,-19,"(-25, -10]",586,-470,-177,116,4,2677
3,-4,60.5,778.50,-16.19,3.80,4.90,20130316,506,2013-03-16 05:06:00,5,2013-03-16 05:00:00,-19,"(-25, -10]",562,-418,-137,144,6,2677
4,-4,60.5,578.11,-15.40,7.43,12.92,20130316,508,2013-03-16 05:08:00,5,2013-03-16 05:00:00,-19,"(-25, -10]",569,-450,-166,119,8,2677


In [7]:
saidDF = velGmagDF.groupby( ["date", "normMLT"] ).max()
saidDF.head()

MLAT   vSaps  azim  vMagnErr  azimErr     dtStr  \
date                normMLT                                                    
2011-02-05 02:30:00 -5       63.5  704.08 -7.26      2.88    16.26  20110205   
                    -4       63.0  704.08 -7.26      2.88    16.26  20110205   
                    -3       63.0  635.56 -5.38      3.02    18.32  20110205   
2011-02-05 02:32:00 -5       63.5  615.93 -4.76      3.40    20.53  20110205   
                    -4       63.0  615.93 -4.76      4.17    22.89  20110205   

                             tmStr  hour            dst_date  dst_index  \
date                normMLT                                               
2011-02-05 02:30:00 -5         230     2 2011-02-05 02:00:00        -50   
                    -4         230     2 2011-02-05 02:00:00        -50   
                    -3         230     2 2011-02-05 02:00:00        -50   
2011-02-05 02:32:00 -5         232     2 2011-02-05 02:00:00        -50   
                    -4         232     2 2011-02-05 02:00:00        -50   

                                dst_bin   AE  AL  AO   AU  minute  count  
date                normMLT                                               
2011-02-05 02:30:00 -5       (-75, -50]  202 -84  17  118      30   2275  
                    -4       (-75, -50]  202 -84  17  118      30   3091  
                    -3       (-75, -50]  202 -84  17  118      30   3529  
2011-02-05 02:32:00 -5       (-75, -50]  247 -92  32  155      32   2275  
                    -4       (-75, -50]  247 -92  32  155      32   3091